In [1]:

from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
#import libraries
import warnings 
warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import scipy.io as spio
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
import math 
import re 
import pandas as pd
import os
import glob
from os.path import expanduser
import datetime
import statistics
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)


In [3]:
def color_negative_red(val):
    color = 'red' if val > 110 else 'black'
    return 'color: %s' % color


In [4]:
class Keypoint:
    tag = ""
    parent = ['']
    child = ['']
    point = None

    def __init__(self,tag=None,parent=None,child=None,point=None):
        if tag is not None:            
            self.tag = tag
        if parent is not None:
            self.parent = parent
        if child is not None:
            self.child = child
        if point is not None:
            self.point = point


In [5]:
class Skeleton:
    keypoints = [Keypoint() for i in range(17)]
    tag2id = {
        "shoulderCenter" : 0,
        "head" : 1,
        "shoulderLeft" : 2,
        "elbowLeft" : 3,
        "handLeft" : 4,
        "shoulderRight" : 5,
        "elbowRight" : 6,
        "handRight" : 7,
        "hipCenter" : 8,
        "hipLeft" : 9,
        "kneeLeft" : 10,
        "ankleLeft" : 11,
        "footLeft" : 12,
        "hipRight" : 13,
        "kneeRight" : 14,
        "ankleRight" : 15,
        "footRight" : 16,
    }
    keypoints[tag2id["shoulderCenter"]] = Keypoint("shoulderCenter",[''],['head','shoulderLeft','shoulderRight','hipCenter'])
    keypoints[tag2id["head"]] = Keypoint("head",['shoulderCenter'],[''])
    keypoints[tag2id["shoulderLeft"]] = Keypoint("shoulderLeft",['shoulderCenter'],['elbowLeft'])
    keypoints[tag2id["elbowLeft"]] = Keypoint("elbowLeft",['shoulderLeft'],['handLeft'])
    keypoints[tag2id["handLeft"]] = Keypoint("handLeft",['elbowLeft'],[''])
    keypoints[tag2id["shoulderRight"]] = Keypoint("shoulderRight",['shoulderCenter'],['elbowRight'])
    keypoints[tag2id["elbowRight"]] = Keypoint("elbowRight",['shoulderRight'],['handRight'])
    keypoints[tag2id["handRight"]] = Keypoint("handRight",['elbowRight'],[''])
    keypoints[tag2id["hipCenter"]] = Keypoint("hipCenter",['shoulderCenter'],['hipLeft','hipRight'])
    keypoints[tag2id["hipLeft"]] = Keypoint("hipLeft",['shoulderCenter'],['kneeLeft'])
    keypoints[tag2id["kneeLeft"]] = Keypoint("kneeLeft",['hipLeft'],['ankleLeft'])
    keypoints[tag2id["ankleLeft"]] = Keypoint("ankleLeft",['kneeLeft'],['footLeft'])
    keypoints[tag2id["footLeft"]] = Keypoint("footLeft",['ankleLeft'],[''])
    keypoints[tag2id["hipRight"]] = Keypoint("hipRight",['shoulderCenter'],['kneeRight'])
    keypoints[tag2id["kneeRight"]] = Keypoint("kneeRight",['hipRight'],['ankleRight'])
    keypoints[tag2id["ankleRight"]] = Keypoint("ankleRight",['kneeRight'],['footRight'])
    keypoints[tag2id["footRight"]] = Keypoint("footRight",['ankleRight'],[''])

    def __init__(self,keyp_map=None):
        if keyp_map is not None:
            for i in range(len(keyp_map)):
                tag = keyp_map.keys()[i]
                self.keypoints[self.tag2id[tag]].point = keyp_map[tag] 

    def getKeypoint(self,keyp_tag):
        return self.keypoints[self.tag2id[keyp_tag]].point

    def getChild(self,keyp_tag):
        return self.keypoints[self.tag2id[keyp_tag]].child

    def getParent(self,keyp_tag):
        return self.keypoints[self.tag2id[keyp_tag]].parent

    def getTransformation(self):
        sagittal = None
        coronal = None
        transverse = None
        T = np.eye(4,4)
        if self.getKeypoint("shoulderLeft") is not None:
            if self.getKeypoint("shoulderRight") is not None:
                sagittal = self.getKeypoint("shoulderLeft")[0]-self.getKeypoint("shoulderRight")[0]
                sagittal = sagittal/np.linalg.norm(sagittal)
        if self.getKeypoint("shoulderCenter") is not None:
            if self.getKeypoint("hipLeft") is not None:
                if self.getKeypoint("hipRight") is not None:
                    transverse = self.getKeypoint("shoulderCenter")[0]-0.5*(self.getKeypoint("hipLeft")[0]+self.getKeypoint("hipRight")[0])
                    transverse = transverse/np.linalg.norm(transverse)
        if self.getKeypoint("shoulderCenter") is not None:
            pSC = self.getKeypoint("shoulderCenter")[0]

        if sagittal is not None:
            if coronal is not None:
                coronal = np.cross(sagittal,transverse)
                T[0,0]=coronal[0]
                T[1,0]=coronal[1]
                T[2,0]=coronal[2]
                T[0,1]=sagittal[0]
                T[1,1]=sagittal[1]
                T[2,1]=sagittal[2]
                T[0,2]=transverse[0]
                T[1,2]=transverse[1]
                T[2,2]=transverse[2]
                T[0,3]=pSC[0]
                T[1,3]=pSC[1]
                T[2,3]=pSC[2]
                T[3,3]=1
        return T

    def show(self):
        for i in range(len(self.keypoints)):
            k = self.keypoints[i]
            print "keypoint[", k.tag, "]", "=", k.point


In [6]:
class Exercise:
    name = ""
    typee = ""
    metrics = []

class Tug(Exercise):
    name = "tug"
    typee = "test"
    metrics = ["ROM_0","ROM_1","ROM_2","ROM_3","ROM_4","ROM_5","step_0"]
    result = []
    month_res = {
        0: [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: [],
        6: [],
        7: [],
        8: [],
        9: [],
        10: [],
        11: []
    }

    def __init__(self,month,result):
        self.result = result
        self.month_res[month] = result

    def getResult(self,month):
        return self.month_res[month]

class Abduction(Exercise):
    name = "abduction"
    typee = "rehabilitation"
    metrics = ["ROM_0"]    
    result = []
    month_res = {
        0: [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: [],
        6: [],
        7: [],
        8: [],
        9: [],
        10: [],
        11: []
    }
    def __init__(self,name,monthi,result):
        self.name = name
        self.result = result
        self.month_res[monthi] = result

    def getResult(self,month):
        return self.month_res[month]

class Internal_Rotation(Exercise):
    name = "internal_rotation"
    typee = "rehabilitation"
    metrics = ["ROM_0"]    
    result = []
    month_res = {
        0: [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: [],
        6: [],
        7: [],
        8: [],
        9: [],
        10: [],
        11: []
    }
    def __init__(self,name,monthi,result):
        self.name = name
        self.result = result
        self.month_res[monthi] = result

    def getResult(self,month):
        return self.month_res[month]

class External_Rotation(Exercise):
    name = "external_rotation"
    typee = "rehabilitation"
    metrics = ["ROM_0"]    
    result = []
    month_res = {
        0: [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: [],
        6: [],
        7: [],
        8: [],
        9: [],
        10: [],
        11: []
    }
    def __init__(self,name,monthi,result):
        self.name = name
        self.result = result
        self.month_res[monthi] = result

    def getResult(self,month):
        return self.month_res[month]

class Reaching(Exercise):
    name = "reaching"
    typee = "rehabilitation"
    metrics = ["EP_0"]    
    result = []
    month_res = {
        0: [],
        1: [],
        2: [],
        3: [],
        4: [],
        5: [],
        6: [],
        7: [],
        8: [],
        9: [],
        10: [],
        11: []
    }
    def __init__(self,name,monthi,result):
        self.name = name
        self.result = result
        self.month_res[monthi] = result

    def getResult(self,month):
        return self.month_res[month]




In [7]:
class Metric:
    name = ''

    def __init__(self,name):
        self.name = name

class Rom(Metric):
    name = "ROM"
    tagjoint = ""
    refjoint = ""
    refdir = []
    tagplane = ""
    def __init__(self,tagjoint,refjoint,refdir,tagplane):
        self.tagjoint = tagjoint
        self.refjoint = refjoint
        self.refdir = refdir
        self.tagplane = tagplane

    def compute(self,skeleton):    
        #joint ref and child
        tj = skeleton.getKeypoint(self.tagjoint)
        tagchild = skeleton.getChild(self.tagjoint)[0]
        cj = skeleton.getKeypoint(tagchild)

        xrj = []
        yrj = []
        zrj = []
        if self.refjoint != "":
            rj = skeleton.getKeypoint(self.refjoint)
            xrj = rj[:,0]
            yrj = rj[:,1]
            zrj = rj[:,2]

        #compute metric
        x=tj[:,0]
        y=tj[:,1]
        z=tj[:,2]

        xchild=cj[:,0]
        ychild=cj[:,1]
        zchild=cj[:,2]

        #plane over which we want to evaluate the metric
        plane = np.zeros(3)
        if tagplane == "coronal":
            plane[0] = 1.0
        if tagplane == "sagittal":
            plane[1] = 1.0
        if tagplane == "transverse":
            plane[2] = 1.0

        #project v1 on the right plane
        invT = np.linalg.inv(skeleton.getTransformation())
        cosRom = []        
        for i in range(len(x)):
            temp_ref = np.array([x[i],y[i],z[i],1])
            temp_child = np.array([xchild[i],ychild[i],zchild[i],1])

            transf_ref = np.inner(invT,temp_ref) 
            transf_child = np.inner(invT,temp_child) 
            vprocess = transf_child-transf_ref
            vprocess = np.delete(vprocess,3)

            dist = np.dot(vprocess,np.transpose(plane))   
            vprocess = vprocess-dist*plane

            n1 = np.linalg.norm(vprocess)
            if(n1>0):
                vprocess = vprocess/n1                

            if len(xrj)>0:
                temp_refjoint = np.array([xrj[i],yrj[i],zrj[i],1])
                transf_refjoint = np.inner(invT,temp_refjoint)
                vecref = transf_ref - transf_refjoint
                ref = np.delete(vecref,3)
            else:    
                n2 = np.linalg.norm(self.refdir)
                if(n2>0):
                    self.refdir = self.refdir/n2
                ref = self.refdir

            dotprod = np.dot(vprocess,np.transpose(ref))
            cosRom.append(dotprod)


        rom_value = np.arccos(cosRom)
        result = rom_value *(180/math.pi)
        return result


class Step(Metric):
    name = "step"
    num = []
    den = []
    tstart = 0.0
    tend = 0.0
    steplen = []
    nsteps = 0
    cadence = 0.0
    speed = 0.0
    ex_time = 0.0
    filtered_distfeet = []
    strikes = []
    def __init__(self,num,den,tstart,tend,step_thresh):
        self.num = num
        self.den = den
        self.tstart = tstart
        self.tend = tend
        self.step_thresh = step_thresh

    def compute(self,skeleton):
        alj = skeleton.getKeypoint("ankleLeft")
        arj = skeleton.getKeypoint("ankleRight")
        sagittal = None
        if skeleton.getKeypoint("shoulderLeft") is not None:
            if skeleton.getKeypoint("shoulderRight") is not None:
                sagittal = skeleton.getKeypoint("shoulderLeft")[0]-skeleton.getKeypoint("shoulderRight")[0]
                sagittal = sagittal/np.linalg.norm(sagittal)
        distfeet = []
        for i in range(len(alj)):
            v = alj[i,:]-arj[i,:]
            dist = np.dot(v,np.transpose(sagittal))   
            v = v-dist*sagittal
            distfeet.append(np.linalg.norm(v))
        self.filtered_distfeet = scipy.signal.filtfilt(self.num,self.den,distfeet)
        self.strikes,_ = scipy.signal.find_peaks(self.filtered_distfeet,height=self.step_thresh)
        filtered_distfeet_np = np.array(self.filtered_distfeet)
        slen = filtered_distfeet_np[self.strikes]
        self.steplen = statistics.mean(slen)
        self.nsteps = len(self.strikes)
        self.cadence = self.nsteps/(self.tend-self.tstart)
        self.speed = self.steplen*self.cadence
        self.ex_time = self.tend-self.tstart

class EndPoint(Metric):
    name = "EP"
    tagjoint = ""
    refdir = []
    tagplane = ""
    target = []
    trajectories = []
    tstart = 0.0
    tend = 0.0
    speed = 0.0
    ex_time = 0.0
    def __init__(self,tagjoint,refdir,tagplane,target,tstart,tend):
        self.tagjoint = tagjoint
        self.refdir = refdir
        self.tagplane = tagplane
        self.target = target
        self.tstart = tstart
        self.tend = tend

    def compute(self,skeleton):
        self.ex_time = self.tend-self.tstart
        tj = skeleton.getKeypoint(self.tagjoint)
        x = tj[:,0]
        y = tj[:,1]
        z = tj[:,2]

        plane = np.zeros(3)
        if tagplane == "coronal":
            plane[0] = 1.0
        if tagplane == "sagittal":
            plane[1] = 1.0
        if tagplane == "transverse":
            plane[2] = 1.0
        invT = np.linalg.inv(skeleton.getTransformation())
        self.trajectories = np.zeros([len(x),3])
        for i in range(len(x)):
            temp_jnt = np.array([x[i],y[i],z[i],1])   
            transf_jnt = np.inner(invT,temp_jnt) 
            v = np.delete(transf_jnt,3)
            dist = np.dot(v,np.transpose(plane))      
            v = v-dist*plane
            self.trajectories[i,0]=v[0]
            self.trajectories[i,1]=v[1]
            self.trajectories[i,2]=v[2]

        vel = np.zeros([len(x),3])
        vel[:,0] = np.gradient(self.trajectories[:,0])/self.ex_time
        vel[:,1] = np.gradient(self.trajectories[:,1])/self.ex_time
        vel[:,2] = np.gradient(self.trajectories[:,2])/self.ex_time

        self.speed = 0.0
        for i in range(len(x)):
            self.speed = self.speed + np.linalg.norm([vel[i,0],vel[i,1],vel[i,2]])
        self.speed = self.speed/len(x)


In [8]:
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict



## Dati anagrafici 

In [9]:

#load file 
home = expanduser("~")
pth = home + '/.local/share/yarp/contexts/motionAnalyzer'
files = glob.glob(os.path.join(pth, '*.mat'))
lastfile = max(files, key=os.path.getctime)
print lastfile

#print personal data
i = [pos for pos, char in enumerate(lastfile) if char == "-"]
i1 = i[-3]
i2 = i[-2]
name = lastfile[i1+1:i2]
surname = ""
age = ""

personaldata = []
personaldata.append(name)
personaldata.append(surname)
personaldata.append(age)        
table = pd.DataFrame(personaldata) 
table.rename(index={0:"Nome",1:"Cognome",2:"Età"},  columns={0:"Paziente"}, inplace=True)
display(table) 

/home/vvasco/.local/share/yarp/contexts/motionAnalyzer/user-#1-tug-0.mat


,Paziente
Nome,#1
Cognome,
Età,


In [10]:

data = []
ctime = []
filename = []
tagex = []
files.sort(key=os.path.getctime)
for fi in files:
    i = [pos for pos, char in enumerate(fi) if char == "-"]
    i1 = i[-3]
    i2 = i[-2]
    i3 = i[-1]
    namei = fi[i1+1:i2]
    if namei == name:
        filename.append(fi)
        data.append(loadmat(fi)) #data.append(scipy.io.loadmat(fi))
        tagex.append(fi[i2+1:i3])
        ctime.append(os.path.getctime(fi))   

In [11]:
time = []
month = []
exercises = []
ex_names = []
#count how many exercise of the same type were performed at that month
countexmonth = {
    "tug" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "abduction_left" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "internal_rotation_left" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "external_rotation_left" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "reaching_left" : [0,0,0,0,0,0,0,0,0,0,0,0]
}

for i in range(len(data)):

    datai = data[i]
    time.append(datai['Time_samples'])
    monthi = datetime.date.fromtimestamp(ctime[i]).month-1
    month.append(monthi)
    shoulderCenter = datai['Keypoints']['shoulderCenter']
    head = datai['Keypoints']['head']
    shoulderLeft = datai['Keypoints']['shoulderLeft']
    shoulderRight = datai['Keypoints']['shoulderRight']
    elbowLeft = datai['Keypoints']['elbowLeft']
    handLeft = datai['Keypoints']['handLeft']
    elbowRight = datai['Keypoints']['elbowRight']
    handRight = datai['Keypoints']['handRight']
    hipLeft = datai['Keypoints']['hipLeft']
    hipRight = datai['Keypoints']['hipRight']
    ankleLeft = datai['Keypoints']['ankleLeft']
    ankleRight = datai['Keypoints']['ankleRight']
    kneeLeft = datai['Keypoints']['kneeLeft']
    kneeRight = datai['Keypoints']['kneeRight']
    footLeft = datai['Keypoints']['footLeft']
    footRight = datai['Keypoints']['footRight']
    hipCenter = datai['Keypoints']['hipCenter']

    key_pam = {
        "shoulderCenter" : shoulderCenter,
        "head" : head,
        "shoulderLeft" : shoulderLeft,
        "shoulderRight" : shoulderRight,
        "elbowLeft" : elbowLeft,
        "handLeft" : handLeft,
        "elbowRight" : elbowRight,
        "handRight" : handRight,
        "hipLeft" : hipLeft,
        "hipRight" : hipRight,
        "ankleLeft" : ankleLeft,
        "ankleRight" : ankleRight,
        "kneeLeft" : kneeLeft,
        "kneeRight" : kneeRight,
        "footLeft" : footLeft,
        "footRight" : footRight,
        "hipCenter" : hipCenter
    }
    s=Skeleton(key_pam)
    #s.show()

    exname = datai["Exercise"]["name"]
    exname = re.sub(r'[^\w]','',exname)
    ex_names.append(exname)
    result_singleexercise = []
    allmet = datai["Exercise"]["metrics"]
    metrics = allmet.keys()
    for j in range(len(metrics)):
        metname = metrics[j]
        if "ROM" in metname:
            tagjoint = allmet[metname]["tag_joint"]
            tagjoint = re.sub(r'[^\w]', '',tagjoint)
            refjoint = allmet[metname]["ref_joint"]
            refjoint = re.sub(r'[^\w]', '',refjoint)
            if type(refjoint) is np.ndarray:
                refjoint = ""
            refdir = allmet[metname]["ref_dir"]
            tagplane = allmet[metname]["tag_plane"]
            tagplane = re.sub(r'[^\w]', '',tagplane)
            rom = Rom(tagjoint,refjoint,refdir,tagplane)
            result_singleexercise.append((rom,rom.compute(s)))

        if "step" in metname:
            num = allmet[metname]["num"]
            den = allmet[metname]["den"]
            tstart = allmet[metname]["tstart"]
            tend = allmet[metname]["tend"]
            step_thresh = allmet[metname]["step_thresh"]
            step = Step(num,den,tstart,tend,step_thresh)
            step.compute(s)
            stepmet = [step.steplen,step.nsteps,step.cadence,step.speed,step.ex_time,
                       step.filtered_distfeet,step.strikes]
            result_singleexercise.append((step,stepmet))

        if "EP" in metname:
            tagjoint = allmet[metname]["tag_joint"]
            tagjoint = re.sub(r'[^\w]', '',tagjoint)
            refdir = allmet[metname]["ref_dir"]
            tagplane = allmet[metname]["tag_plane"]
            tagplane = re.sub(r'[^\w]', '',tagplane)
            target = allmet[metname]["target"]
            tstart = allmet[metname]["tstart"]
            tend = allmet[metname]["tend"]
            ep = EndPoint(tagjoint,refdir,tagplane,target,tstart,tend)
            ep.compute(s)
            result_singleexercise.append((ep,ep.trajectories))

    if exname == "tug":
        ex = Tug(monthi,result_singleexercise)
    if "abduction" in exname:
        ex = Abduction(exname,monthi,result_singleexercise)
    if "internal_rotation" in exname:
        ex = Internal_Rotation(exname,monthi,result_singleexercise)
    if "external_rotation" in exname:
        ex = External_Rotation(exname,monthi,result_singleexercise)
    if "reaching" in exname:
        ex = Reaching(exname,monthi,result_singleexercise)

    countexmonth[exname][monthi] = 1 + countexmonth[exname][monthi]

    exercises.append(ex)   


 
## Report della seduta odierna
Il paziente ha svolto il seguente esercizio: 

In [12]:

print exname.encode('ascii') 

tug


 
on: 

In [13]:

now = datetime.datetime.now()
print now 

2020-05-06 14:25:57.134510


 
Di seguito si riporta il grafico:

In [14]:
lastsess_time = time[-1]
lastsess_result = exercises[-1].result
lastsess_res_step = []

%matplotlib inline
%matplotlib inline
for i in range(len(lastsess_result)):
    lastsess_met,lastsess_resi = lastsess_result[i]
    lastsess_metname = lastsess_met.name
    ################
    #     ROM      #
    ################
    if lastsess_metname == "ROM":
        lastsess_metjoint = lastsess_met.tagjoint
        trace1 = go.Scatter(
            x=lastsess_time,y=lastsess_resi,
            mode='lines',
            line=dict(
                color='blue',
                width=3
            ),
            name='Andamento Reale'
        )
        data = [trace1]
        layout = dict(
            width=750,
            height=600,
            autosize=False,
            title='Range of Motion '+lastsess_metjoint,
            font=dict(family='Courier New, monospace', size=22, color='black'),
            xaxis=dict(
                title='time [s]',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            ),
            yaxis=dict(
                title='ROM [gradi]',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            )
        )
        fig = dict(data=data, layout=layout)
        iplot(fig)

    ################
    #     STEP     #
    ################
    if lastsess_metname == "step":
        dist=lastsess_resi[5]
        strikes=lastsess_resi[6]
        trace1 = go.Scatter(
            x=lastsess_time,y=dist,
            mode='lines',
            line=dict(
                color='blue',
                width=3
            ),
            name='Distanza tra i piedi'
        )
        trace2 = go.Scatter(
            x=lastsess_time[strikes],y=dist[strikes],
            mode='markers',
            marker=dict(
                color='red',
                size=10
            ),
            name='Appoggio tallone'
        )
        data = [trace1,trace2]
        layout = dict(
            width=750,
            height=600,
            autosize=False,
            title='Distanza tra i piedi',
            font=dict(family='Courier New, monospace', size=22, color='black'),
            xaxis=dict(
                title='time [s]',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            ),
            yaxis=dict(
                title='dist [m]',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            )
        )
        fig = dict(data=data, layout=layout)
        iplot(fig)

        lastsess_res_step.append(lastsess_resi)
        tablestep = pd.DataFrame(lastsess_res_step[0][0:4])
        tablestep.rename(index={0:"Lunghezza passo [m]",1:"Numero di passi",2:"Cadenza [steps/s]",
                                3:"Velocità [m/s]",4:"Tempo di esecuzione [s]"},
                         columns={0:"Analisi cammino"}, inplace=True)
        display(tablestep)

    ################
    #      EP      #
    ################
    if lastsess_metname == "EP":
        target = lastsess_met.target
        trace1 = go.Scatter3d(
            x=lastsess_resi[:,0], y=lastsess_resi[:,1], z=lastsess_resi[:,2],
            mode = 'lines',
            line=dict(
                color='blue',
                width=3
            ),
            name = 'Traiettorie'
        )
        trace2 = go.Scatter3d(
            x=[target[0]], y=[target[1]], z=[target[2]],
            mode = 'markers',
            marker=dict(
                color='red',
                size=5
            ),
            name = 'Target da raggiungere'
        )
        data = [trace1, trace2]    
        layout = dict(
            margin=dict(
                l=0,
                r=0,
                b=0
            ),
            title='Traiettorie End-point',
            font=dict(family='Courier New, monospace', size=22, color='black'),
            scene=dict(
                xaxis=dict(
                    title='x [cm]',
                    titlefont=dict(
                        family='Courier New, monospace',
                        size=18,
                        color='#7f7f7f'
                    ),
                    tickfont=dict(
                        family='Courier New, monospace',
                        size=14,
                        color='#7f7f7f'
                    ),
                    gridcolor='rgb(255, 255, 255)',
                    zerolinecolor='rgb(255, 255, 255)',
                    showbackground=True,
                    backgroundcolor='rgb(230, 230,230)'
                ),
                yaxis=dict(
                    title='y [cm]',
                    titlefont=dict(
                        family='Courier New, monospace',
                        size=18,
                        color='#7f7f7f'
                    ),
                    tickfont=dict(
                        family='Courier New, monospace',
                        size=14,
                        color='#7f7f7f'
                    ),
                    gridcolor='rgb(255, 255, 255)',
                    zerolinecolor='rgb(255, 255, 255)',
                    showbackground=True,
                    backgroundcolor='rgb(230, 230,230)'
                ),
                zaxis=dict(
                    title='z [cm]',
                    titlefont=dict(
                        family='Courier New, monospace',
                        size=18,
                        color='#7f7f7f'
                    ),
                    tickfont=dict(
                        family='Courier New, monospace',
                        size=14,
                        color='#7f7f7f'
                    ),
                    gridcolor='rgb(255, 255, 255)',
                    zerolinecolor='rgb(255, 255, 255)',
                    showbackground=True,
                    backgroundcolor='rgb(230, 230,230)'
                ),
                camera=dict(
                    up=dict(
                        x=0,
                        y=0,
                        z=1
                    ),
                    eye=dict(
                        x=-1.7428,
                        y=1.0707,
                        z=0.7100,
                    )
                ),
                aspectratio = dict( x=1, y=1, z=0.7 ),
                aspectmode = 'manual'
            ),
        )
        fig = dict(data=data, layout=layout)
        iplot(fig)


,Analisi cammino
Lunghezza passo [m],0.355625
Numero di passi,2.000000
Cadenza [steps/s],0.131253
Velocità [m/s],0.046677


In [15]:
if exname == 'tug':

    table_tug = pd.DataFrame([['Normale mobilità'],['Buona mobilita'],['Ausili cammino'],['Rischio caduta']],
             index=['< 10 s','< 20 s','< 30 s','>= 30 s'],
             columns=['Tabella TUG'])
    display(table_tug)

    time_score = lastsess_res_step[0][4]
    print "Il test è stato svolto in",round(time_score,2),"s"
    if time_score < 10:
        evaluation = 'Normale mobilità'
        print "La valutazione è [1;30;42m",evaluation
    elif time_score < 20:
        evaluation = 'Buona mobilità, non necessita ausili'
        print "La valutazione è [1;30;42m",evaluation
    elif time_score < 30:
        evaluation = 'Necessita ausili cammino'
        print "La valutazione è [1;30;43m",evaluation
    elif time_score >= 30:
        evaluation = 'Rischio caduta'
        print "La valutazione è [1;30;41m",evaluation


,Tabella TUG
< 10 s,Normale mobilità
< 20 s,Buona mobilita
< 30 s,Ausili cammino
>= 30 s,Rischio caduta


Il test è stato svolto in 15.24 s
La valutazione è  Buona mobilità, non necessita ausili


 
## Andamento stato clinico

 
Gli esercizi svolti dal paziente nei mesi in analisi sono raggruppati come segue:

In [16]:
labels = ["reaching_left","abduction_left","internal-rotation_left","external-rotation_left","timed-up-and-go"]
values = [tagex.count("reaching_left"), tagex.count("abduction_left"), tagex.count("internal_rotation_left"), 
          tagex.count("external_rotation_left"), tagex.count("tug")]
colors = ['#FEBFB3', '#E1396C', '#96D38C', '#D0F9B1']

trace = go.Pie(labels=labels, values=values,
               #hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20),
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)),
               hoverinfo="label+percent+value",
               hole=0.3
              )
layout = go.Layout(
        title="Performed exercises",
)
data = [trace]
fig = go.Figure(data=data,layout=layout)
iplot(fig)   

 
Di seguito, si riportano gli andamenti delle metriche del paziente, raggruppati per mese: 

In [17]:
keyp2rommax = {
    "shoulderCenter" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "head" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "shoulderLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "elbowLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "handLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "shoulderRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "elbowRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "handRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipCenter" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "kneeLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "ankleLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "footLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "kneeRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "ankleRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "footRight" : [0,0,0,0,0,0,0,0,0,0,0,0]
}
keyp2rommin = {
    "shoulderCenter" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "head" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "shoulderLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "elbowLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "handLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "shoulderRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "elbowRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "handRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipCenter" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "kneeLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "ankleLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "footLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "kneeRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "ankleRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "footRight" : [0,0,0,0,0,0,0,0,0,0,0,0]
}
countrommonth = {
    "shoulderCenter" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "head" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "shoulderLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "elbowLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "handLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "shoulderRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "elbowRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "handRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipCenter" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "kneeLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "ankleLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "footLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "kneeRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "ankleRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "footRight" : [0,0,0,0,0,0,0,0,0,0,0,0]
}
keyp2rommax_avg = {
    "shoulderCenter" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "head" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "shoulderLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "elbowLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "handLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "shoulderRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "elbowRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "handRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipCenter" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "kneeLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "ankleLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "footLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "kneeRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "ankleRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "footRight" : [0,0,0,0,0,0,0,0,0,0,0,0]
}
keyp2rommin_avg = {
    "shoulderCenter" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "head" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "shoulderLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "elbowLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "handLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "shoulderRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "elbowRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "handRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipCenter" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "kneeLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "ankleLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "footLeft" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "hipRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "kneeRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "ankleRight" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "footRight" : [0,0,0,0,0,0,0,0,0,0,0,0]
}

stepmonth = {
    "steplen" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "numsteps" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "cadence" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "speed" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "time" : [0,0,0,0,0,0,0,0,0,0,0,0]
}
countstepmonth = [0,0,0,0,0,0,0,0,0,0,0,0]

endpointmonth = {
    "time" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "speed" : [0,0,0,0,0,0,0,0,0,0,0,0]
}
countendpointmonth = [0,0,0,0,0,0,0,0,0,0,0,0]

for i in range(len(exercises)):
    exnamei = exercises[i].name
    for monthi in range(12):
        if countexmonth[exnamei][monthi] != 0:
            res_exi = exercises[i].getResult(monthi)
            for m in range(len(res_exi)): 
                single_metric,result_single_metric = res_exi[m]
                single_metric_name = single_metric.name
                if single_metric_name == "ROM":
                    maxromj = max(result_single_metric)
                    minromj = min(result_single_metric)
                    tagjoint = single_metric.tagjoint
                    keyp2rommax[tagjoint][monthi] = maxromj + keyp2rommax[tagjoint][monthi]
                    keyp2rommin[tagjoint][monthi] = minromj + keyp2rommin[tagjoint][monthi]
                    countrommonth[tagjoint][monthi] = 1 + countrommonth[tagjoint][monthi]           

                if single_metric_name == "step":
                    stepmonth["steplen"][monthi] = single_metric.steplen + stepmonth["steplen"][monthi]
                    stepmonth["numsteps"][monthi] = single_metric.nsteps + stepmonth["numsteps"][monthi]
                    stepmonth["cadence"][monthi] = single_metric.cadence + stepmonth["cadence"][monthi]
                    stepmonth["speed"][monthi] = single_metric.speed + stepmonth["speed"][monthi]
                    stepmonth["time"][monthi] = single_metric.ex_time + stepmonth["time"][monthi]
                    countstepmonth[monthi] = 1 + countstepmonth[monthi]

                if single_metric_name == "EP":
                    endpointmonth["time"][monthi] = single_metric.ex_time + endpointmonth["time"][monthi]
                    endpointmonth["speed"][monthi] = single_metric.speed + endpointmonth["speed"][monthi]
                    countendpointmonth[monthi] = 1 + countendpointmonth[monthi]



In [ ]:
counted_exmonth = {
    "tug" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "abduction_left" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "internal_rotation_left" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "external_rotation_left" : [0,0,0,0,0,0,0,0,0,0,0,0],
    "reaching_left" : [0,0,0,0,0,0,0,0,0,0,0,0]
}
for i in range(len(exercises)):
    exnamei = exercises[i].name

    for monthi in range(12):  
        if countexmonth[exnamei][monthi] != 0:
            if counted_exmonth[exnamei][monthi] < 1:       
                res_exi = exercises[i].getResult(monthi)
                for m in range(len(res_exi)): 
                    single_metric,result_single_metric = res_exi[m]
                    single_metric_name = single_metric.name

                    if single_metric_name == "ROM":
                        tagjoint = single_metric.tagjoint
                        if countrommonth[tagjoint][monthi] != 0:
                            keyp2rommax_avg[tagjoint][monthi] = keyp2rommax[tagjoint][monthi]/countrommonth[tagjoint][monthi]
                            keyp2rommin_avg[tagjoint][monthi] = keyp2rommin[tagjoint][monthi]/countrommonth[tagjoint][monthi]


                    if single_metric_name == "step":
                        if countstepmonth[monthi] != 0:
                            stepmonth["steplen"][monthi] = stepmonth["steplen"][monthi]/countstepmonth[monthi]
                            stepmonth["numsteps"][monthi] = stepmonth["numsteps"][monthi]/countstepmonth[monthi]
                            stepmonth["cadence"][monthi] = stepmonth["cadence"][monthi]/countstepmonth[monthi]
                            stepmonth["speed"][monthi] = stepmonth["speed"][monthi]/countstepmonth[monthi]
                            stepmonth["time"][monthi] = stepmonth["time"][monthi]/countstepmonth[monthi]   

                    if single_metric_name == "EP":
                        if countendpointmonth[monthi] != 0:
                            endpointmonth["time"][monthi] = endpointmonth["time"][monthi]/countendpointmonth[monthi]
                            endpointmonth["speed"][monthi] = endpointmonth["speed"][monthi]/countendpointmonth[monthi]

                counted_exmonth[exnamei][monthi] = 1



In [19]:
title = "Risultati del Timed Up and Go"
msg1 = HTML('<font size=5><b>'+title+'</b></font>')
msg2 = HTML('<font size=5,font color=gray,font >'+title+'</font>')
if sum(counted_exmonth['tug'])>0:
    display(msg1);
else:
    display(msg2); 

In [20]:
allmonths = [1,2,3,4,5,6,7,8,9,10,11,12]
counted_ex = {
    "tug" : 0,
    "abduction_left" : 0,
    "internal_rotation_left" : 0,
    "external_rotation_left" : 0,
    "reaching_left" : 0
}
for i in range(len(exercises)):
    exnamei = exercises[i].name
    res_exi = exercises[i].result
    if counted_ex[exnamei] < 1:

        #############################
        #      Results for TUG      #
        #############################
        if exnamei == "tug":
            counted_ex[exnamei] = 1
            step_month_table = pd.DataFrame.from_dict(stepmonth,orient='index',
                                                      columns=['Jan','Feb','Mar','Apr','May','Jun',
                                                           'Jul','Aug','Sep','Oct','Nov','Dec'])
            step_month_table.index = ["Numero di passi","Velocità [m/s]","Lunghezza passo [m]",
                                      "Cadenza [steps/s]","Tempo di esecuzione [s]"]
            display(step_month_table)

            for m in range(len(res_exi)): 
                single_metric,result_single_metric = res_exi[m]
                if single_metric.name == "ROM":
                    tagjoint = single_metric.tagjoint

                    if np.sum(keyp2rommax_avg[tagjoint]) > 0.0:
                        trace1 = go.Bar(
                            x=allmonths, 
                            y=keyp2rommax_avg[tagjoint],
                            name='Massimo valore raggiunto',
                            marker=dict(
                                color='rgb(0,0,255)'
                            )
                        )
                        trace2 = go.Bar(
                            x=allmonths, 
                            y=keyp2rommin_avg[tagjoint],
                            name='Minimo valore raggiunto',
                            marker=dict(
                                color='rgb(255,0,0)'
                                )
                            )
                        layout = go.Layout(
                            title='Parametri Range of Motion',
                            font=dict(family='Courier New, monospace', size=18, color='black'),
                            xaxis=dict(
                                title='Month',
                                titlefont=dict(
                                    family='Courier New, monospace',
                                    size=18,
                                    color='#7f7f7f'
                                ),
                                tickfont=dict(
                                    family='Courier New, monospace',
                                    size=14,
                                    color='#7f7f7f'
                                )
                            ),
                            yaxis=dict(
                                    title='ROM ' + tagjoint + ' [gradi]',
                                    titlefont=dict(
                                        family='Courier New, monospace',
                                        size=18,
                                        color='#7f7f7f'
                                    ),
                            tickfont=dict(
                                        family='Courier New, monospace',
                                        size=14,
                                        color='#7f7f7f'
                                    )
                                ),
                            legend=dict(
                                    bgcolor='rgba(255, 255, 255, 0)',
                                    bordercolor='rgba(255, 255, 255, 0)'
                                ),
                            barmode='group',
                            bargap=0.1,
                            bargroupgap=0.0
                            )
                        data=[trace1,trace2]
                        fig = go.Figure(data=data, layout=layout)
                        iplot(fig)              

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Numero di passi,0,0,0,0,2.000000,0,0,0,0,0,0,0
Velocità [m/s],0,0,0,0,0.046677,0,0,0,0,0,0,0
Lunghezza passo [m],0,0,0,0,0.355625,0,0,0,0,0,0,0
Cadenza [steps/s],0,0,0,0,0.131253,0,0,0,0,0,0,0
Tempo di esecuzione [s],0,0,0,0,15.237801,0,0,0,0,0,0,0


In [21]:
title = "Risultati di Abduzione"
msg1 = HTML('<font size=5><b>'+title+'</b></font>')
msg2 = HTML('<font size=5,font color=gray>'+title+'</font>')
if sum(counted_exmonth['abduction_left'])>0:
    display(msg1);
else:
    display(msg2); 

In [22]:
for i in range(len(exercises)):
    exnamei = exercises[i].name
    res_exi = exercises[i].result
    if counted_ex[exnamei] < 1:

        ###################################
        #      Results for Abduction      #
        ###################################
        if "abduction" in exnamei:
            counted_ex[exnamei] = 1           

            for m in range(len(res_exi)): 
                single_metric,result_single_metric = res_exi[m]
                if single_metric.name == "ROM":
                    tagjoint = single_metric.tagjoint

                    if np.sum(keyp2rommax_avg[tagjoint]) > 0.0:
                        trace1 = go.Bar(
                            x=allmonths, 
                            y=keyp2rommax_avg[tagjoint],
                            name='Massimo valore raggiunto',
                            marker=dict(
                                color='rgb(0,0,255)'
                            )
                        )
                        trace2 = go.Bar(
                            x=allmonths, 
                            y=keyp2rommin_avg[tagjoint],
                            name='Minimo valore raggiunto',
                            marker=dict(
                                color='rgb(255,0,0)'
                                )
                            )
                        layout = go.Layout(
                            title='Parametri Range of Motion',
                            font=dict(family='Courier New, monospace', size=18, color='black'),
                            xaxis=dict(
                                title='Month',
                                titlefont=dict(
                                    family='Courier New, monospace',
                                    size=18,
                                    color='#7f7f7f'
                                ),
                                tickfont=dict(
                                    family='Courier New, monospace',
                                    size=14,
                                    color='#7f7f7f'
                                )
                            ),
                            yaxis=dict(
                                    title='ROM ' + tagjoint + ' [gradi]',
                                    titlefont=dict(
                                        family='Courier New, monospace',
                                        size=18,
                                        color='#7f7f7f'
                                    ),
                            tickfont=dict(
                                        family='Courier New, monospace',
                                        size=14,
                                        color='#7f7f7f'
                                    )
                                ),
                            legend=dict(
                                    bgcolor='rgba(255, 255, 255, 0)',
                                    bordercolor='rgba(255, 255, 255, 0)'
                                ),
                            barmode='group',
                            bargap=0.1,
                            bargroupgap=0.0
                            )
                        data=[trace1,trace2]
                        fig = go.Figure(data=data, layout=layout)
                        iplot(fig)    

In [23]:
title = "Risultati di Rotazione Interna"
msg1 = HTML('<font size=5><b>'+title+'</b></font>')
msg2 = HTML('<font size=5,font color=gray>'+title+'</font>')
if sum(counted_exmonth['internal_rotation_left'])>0:
    display(msg1);
else:
    display(msg2); 

In [24]:
for i in range(len(exercises)):
    exnamei = exercises[i].name
    res_exi = exercises[i].result
    if counted_ex[exnamei] < 1:

        ###################################
        #      Results for Internal      #
        ###################################
        if "internal_rotation" in exnamei:
            counted_ex[exnamei] = 1           

            for m in range(len(res_exi)): 
                single_metric,result_single_metric = res_exi[m]
                if single_metric.name == "ROM":
                    tagjoint = single_metric.tagjoint

                    if np.sum(keyp2rommax_avg[tagjoint]) > 0.0:
                        trace1 = go.Bar(
                            x=allmonths, 
                            y=keyp2rommax_avg[tagjoint],
                            name='Massimo valore raggiunto',
                            marker=dict(
                                color='rgb(0,0,255)'
                            )
                        )
                        trace2 = go.Bar(
                            x=allmonths, 
                            y=keyp2rommin_avg[tagjoint],
                            name='Minimo valore raggiunto',
                            marker=dict(
                                color='rgb(255,0,0)'
                                )
                            )
                        layout = go.Layout(
                            title='Parametri Range of Motion',
                            font=dict(family='Courier New, monospace', size=18, color='black'),
                            xaxis=dict(
                                title='Month',
                                titlefont=dict(
                                    family='Courier New, monospace',
                                    size=18,
                                    color='#7f7f7f'
                                ),
                                tickfont=dict(
                                    family='Courier New, monospace',
                                    size=14,
                                    color='#7f7f7f'
                                )
                            ),
                            yaxis=dict(
                                    title='ROM ' + tagjoint + ' [gradi]',
                                    titlefont=dict(
                                        family='Courier New, monospace',
                                        size=18,
                                        color='#7f7f7f'
                                    ),
                            tickfont=dict(
                                        family='Courier New, monospace',
                                        size=14,
                                        color='#7f7f7f'
                                    )
                                ),
                            legend=dict(
                                    bgcolor='rgba(255, 255, 255, 0)',
                                    bordercolor='rgba(255, 255, 255, 0)'
                                ),
                            barmode='group',
                            bargap=0.1,
                            bargroupgap=0.0
                            )
                        data=[trace1,trace2]
                        fig = go.Figure(data=data, layout=layout)
                        iplot(fig)  

In [25]:
title = "Risultati di Rotazione Esterna"
msg1 = HTML('<font size=5><b>'+title+'</b></font>')
msg2 = HTML('<font size=5,font color=gray>'+title+'</font>')
if sum(counted_exmonth['external_rotation_left'])>0:
    display(msg1);
else:
    display(msg2); 

In [26]:
for i in range(len(exercises)):
    exnamei = exercises[i].name
    res_exi = exercises[i].result
    if counted_ex[exnamei] < 1:

        ###################################
        #      Results for External      #
        ###################################
        if "external_rotation" in exnamei:
            counted_ex[exnamei] = 1           

            for m in range(len(res_exi)): 
                single_metric,result_single_metric = res_exi[m]
                if single_metric.name == "ROM":
                    tagjoint = single_metric.tagjoint

                    if np.sum(keyp2rommax_avg[tagjoint]) > 0.0:
                        trace1 = go.Bar(
                            x=allmonths, 
                            y=keyp2rommax_avg[tagjoint],
                            name='Massimo valore raggiunto',
                            marker=dict(
                                color='rgb(0,0,255)'
                            )
                        )
                        trace2 = go.Bar(
                            x=allmonths, 
                            y=keyp2rommin_avg[tagjoint],
                            name='Minimo valore raggiunto',
                            marker=dict(
                                color='rgb(255,0,0)'
                                )
                            )
                        layout = go.Layout(
                            title='Parametri Range of Motion',
                            font=dict(family='Courier New, monospace', size=18, color='black'),
                            xaxis=dict(
                                title='Mese',
                                titlefont=dict(
                                    family='Courier New, monospace',
                                    size=18,
                                    color='#7f7f7f'
                                ),
                                tickfont=dict(
                                    family='Courier New, monospace',
                                    size=14,
                                    color='#7f7f7f'
                                )
                            ),
                            yaxis=dict(
                                    title='ROM ' + tagjoint + ' [gradi]',
                                    titlefont=dict(
                                        family='Courier New, monospace',
                                        size=18,
                                        color='#7f7f7f'
                                    ),
                            tickfont=dict(
                                        family='Courier New, monospace',
                                        size=14,
                                        color='#7f7f7f'
                                    )
                                ),
                            legend=dict(
                                    bgcolor='rgba(255, 255, 255, 0)',
                                    bordercolor='rgba(255, 255, 255, 0)'
                                ),
                            barmode='group',
                            bargap=0.1,
                            bargroupgap=0.0
                            )
                        data=[trace1,trace2]
                        fig = go.Figure(data=data, layout=layout)
                        iplot(fig)  

In [27]:
title = "Risultati di Reaching"
msg1 = HTML('<font size=5><b>'+title+'</b></font>')
msg2 = HTML('<font size=5,font color=gray>'+title+'</font>')
if sum(counted_exmonth['reaching_left'])>0:
    display(msg1);
else:
    display(msg2); 

In [28]:
for i in range(len(exercises)):
    exnamei = exercises[i].name
    res_exi = exercises[i].result
    if counted_ex[exnamei] < 1:

        ###################################
        #      Results for Reaching      #
        ###################################
        if "reaching" in exnamei:
            counted_ex[exnamei] = 1           

            for m in range(len(res_exi)): 
                single_metric,result_single_metric = res_exi[m]
                if single_metric.name == "EP":
                    trace1 = go.Bar(
                        x=allmonths, 
                        y=endpointmonth["time"],
                        name='Time',
                        marker=dict(
                            color='rgb(0,0,255)'
                        )
                    )
                    layout = go.Layout(
                        title='Parametri Reaching',
                        font=dict(family='Courier New, monospace', size=18, color='black'),
                        xaxis=dict(
                            title='Mese',
                            titlefont=dict(
                                family='Courier New, monospace',
                                size=18,
                                color='#7f7f7f'
                            ),
                            tickfont=dict(
                                family='Courier New, monospace',
                                size=14,
                                color='#7f7f7f'
                            )
                        ),
                        yaxis=dict(
                                title='Tempo di esecuzione [s]',
                                titlefont=dict(
                                    family='Courier New, monospace',
                                    size=18,
                                    color='#7f7f7f'
                                ),
                        tickfont=dict(
                                    family='Courier New, monospace',
                                    size=14,
                                    color='#7f7f7f'
                                )
                            ),
                        legend=dict(
                                bgcolor='rgba(255, 255, 255, 0)',
                                bordercolor='rgba(255, 255, 255, 0)'
                            ),
                        barmode='group',
                        bargap=0.1,
                        bargroupgap=0.0
                        )
                    data=[trace1]
                    fig = go.Figure(data=data, layout=layout)
                    iplot(fig) 

                    trace1 = go.Bar(
                        x=allmonths, 
                        y=endpointmonth["speed"],
                        name='Speed',
                        marker=dict(
                            color='rgb(0,0,255)'
                        )
                    )
                    layout = go.Layout(
                        font=dict(family='Courier New, monospace', size=18, color='black'),
                        xaxis=dict(
                            title='Mese',
                            titlefont=dict(
                                family='Courier New, monospace',
                                size=18,
                                color='#7f7f7f'
                            ),
                            tickfont=dict(
                                family='Courier New, monospace',
                                size=14,
                                color='#7f7f7f'
                            )
                        ),
                        yaxis=dict(
                                title='Velocità [m/s]',
                                titlefont=dict(
                                    family='Courier New, monospace',
                                    size=18,
                                    color='#7f7f7f'
                                ),
                        tickfont=dict(
                                    family='Courier New, monospace',
                                    size=14,
                                    color='#7f7f7f'
                                )
                            ),
                        legend=dict(
                                bgcolor='rgba(255, 255, 255, 0)',
                                bordercolor='rgba(255, 255, 255, 0)'
                            ),
                        barmode='group',
                        bargap=0.1,
                        bargroupgap=0.0
                        )
                    data=[trace1]
                    fig = go.Figure(data=data, layout=layout)
                    iplot(fig)   